## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Qaanaaq,GL,77.4840,-69.3632,41.79,71,96,3.36,overcast clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,75.16,75,67,8.01,broken clouds
2,2,Jamestown,US,42.0970,-79.2353,81.30,91,1,1.01,clear sky
3,3,Rocha,UY,-34.4833,-54.3333,54.21,62,1,4.50,clear sky
4,4,Codrington,AU,-38.2667,141.9667,46.44,96,42,11.95,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 88


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Puerto Ayora,EC,-0.7393,-90.3518,75.16,75,67,8.01,broken clouds
2,2,Jamestown,US,42.0970,-79.2353,81.30,91,1,1.01,clear sky
7,7,Atuona,PF,-9.8000,-139.0333,76.87,74,29,18.16,light rain
8,8,Vila Franca Do Campo,PT,37.7167,-25.4333,75.61,68,2,11.25,clear sky
12,12,Ponta Do Sol,PT,32.6667,-17.1000,79.72,73,52,2.66,broken clouds
20,20,Victoria,HK,22.2855,114.1577,85.32,84,85,9.84,overcast clouds
24,24,Buala,SB,-8.1450,159.5921,77.95,84,100,8.61,overcast clouds
26,26,Georgetown,MY,5.4112,100.3354,83.89,82,20,0.00,few clouds
31,31,Butaritari,KI,3.0707,172.7902,81.95,74,30,8.37,light rain
33,33,Iquitos,PE,-3.7481,-73.2472,77.04,83,75,4.61,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                245
City                   245
Country                245
Lat                    245
Lng                    245
Max Temp               245
Humidity               245
Cloudiness             245
Wind Speed             245
Current Description    245
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [7]:
clean_preferred_cities_df.count()

City_ID                245
City                   245
Country                245
Lat                    245
Lng                    245
Max Temp               245
Humidity               245
Cloudiness             245
Wind Speed             245
Current Description    245
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,
2,Jamestown,US,81.30,clear sky,42.0970,-79.2353,
7,Atuona,PF,76.87,light rain,-9.8000,-139.0333,
8,Vila Franca Do Campo,PT,75.61,clear sky,37.7167,-25.4333,
12,Ponta Do Sol,PT,79.72,broken clouds,32.6667,-17.1000,
20,Victoria,HK,85.32,overcast clouds,22.2855,114.1577,
24,Buala,SB,77.95,overcast clouds,-8.1450,159.5921,
26,Georgetown,MY,83.89,few clouds,5.4112,100.3354,
31,Butaritari,KI,81.95,light rain,3.0707,172.7902,
33,Iquitos,PE,77.04,broken clouds,-3.7481,-73.2472,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   245
Country                245
Max Temp               245
Current Description    245
Lat                    245
Lng                    245
Hotel Name             245
dtype: int64

In [11]:
# No empty rows for Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [12]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
2,Jamestown,US,81.30,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
7,Atuona,PF,76.87,light rain,-9.8000,-139.0333,Villa Enata
8,Vila Franca Do Campo,PT,75.61,clear sky,37.7167,-25.4333,Pestana Bahia Praia
12,Ponta Do Sol,PT,79.72,broken clouds,32.6667,-17.1000,Hotel do Campo


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))